In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score #walidacja krzyżowa(7:30min)

%matplotlib inline 
#w nowych nie potrzebne


In [27]:
#'https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-23.csv'
dft = pd.read_csv('C:\\report_07-23.csv')
dfw = pd.read_csv('C:\\temperatury23072018.csv',delimiter=';',names=['w_stationId','w_eventId','plannedTime','w_qty']) # podwójny backslash
dfw['plannedTime'] = pd.to_datetime(dfw['plannedTime'])
#dfw['volume']=dfw['volume'].map(lambda text: text.replace(',', '.'))
df = pd.concat([dfw,dft],join='outer', sort=True) #keys=['plannedTime','w_dateTime'],sort=True)
#df.head()
#df=pd.DataFrame(pd.merge(dft, dfw, on='plannedTime'))


In [28]:
df

,delay,direction,index,number,plannedTime,seq_num,status,stop,stopName,time_stamp,tripId,vehicleId,w_eventId,w_qty,w_stationId
0,NaN,NaN,NaN,NaN,2018-07-23 05:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,17.11,250190470.0
1,NaN,NaN,NaN,NaN,2018-07-23 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.92,250190470.0
2,NaN,NaN,NaN,NaN,2018-07-23 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.78,250190470.0
3,NaN,NaN,NaN,NaN,2018-07-23 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.81,250190470.0
4,NaN,NaN,NaN,NaN,2018-07-23 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.99,250190470.0
5,NaN,NaN,NaN,NaN,2018-07-23 00:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.95,250190470.0
6,NaN,NaN,NaN,NaN,2018-07-23 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.80,250190470.0
7,NaN,NaN,NaN,NaN,2018-07-23 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.82,250190470.0
8,NaN,NaN,NaN,NaN,2018-07-23 01:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.77,250190470.0
9,NaN,NaN,NaN,NaN,2018-07-23 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B00300S,16.60,250190470.0


In [ ]:
df[ df.tripId == 6351558574044883205 ] # taki select
#trip id jest dostepny dla trasy i jeśli sie go 
#użyje to prognozowanie będzie tylko dla tej podróży

In [ ]:
df.delay.value_counts(normalize=True) 
#40% przyjezdza na czas
#normalizacja pokazuje % w sumie do 100 (1)


In [ ]:
df.delay.hist(bins=15);

In [ ]:
df.delay.describe() #statystyka czasów

In [ ]:
df['plannedTime'] = pd.to_datetime(df['plannedTime'])
#prawidłowa konwersja stringa na time dtypes: datetime64
#inaczej stringowa kolumna jest pokazywana jako object
df[['plannedTime']].info()
df['hour'] = df['plannedTime'].dt.hour #month #day #value_counts()
#dostęp do składowych w dacie

In [ ]:
df['delay_secs'] = df['delay'].map(lambda x: x*60) 
# zeby bylo w sekundach to trzeba pomnożyć przez 60 bo teraz jest w minutach. 
# i wynik wychodzi 0.9 min

df['direction_cat']=df['direction'].factorize()[0]
#factorize przypisze stringom wartości 
#liczbowe w nowej tablicy jako id do tych wartości string (np)
df['vehicleId'].fillna(-1, inplace=True)
#dopisać wartoscia unikowa jaka nie występuje w modelu wogóle (np -1)
df['seq_num'].fillna(-1, inplace=True)
#When inplace=True is passed, the data is renamed in place (it returns nothing), so you'd use:
# df.an_operation(inplace=True)
#When inplace=False is passed (this is the default value, so isn't necessary), 
#performs the operation and returns a copy of the object, so you'd use:

def gen_id_num_direction(x):
    return '{} {}'.format(x['number'], x['direction'])
df['number_direction_id'] = df.apply(gen_id_num_direction, axis=1).factorize()[0]
#zeby dodac wiecej ciekawych cech, można łączyć niektore w całość, 
#np tramwaj+kierunek
#apply działa że x przejeżdza przez każdy wiersz i daje ddostęp do kolumn

def gen_id_stop_direction(x):
    return '{} {}'.format(x['stop'], x['direction'])
df['stop_direction_id'] = df.apply(gen_id_stop_direction, axis=1).factorize()[0]

feats=[
    'number',
    'stop',
    'direction_cat',
    'vehicleId',
    'seq_num',
    'number_direction_id',
    'stop_direction_id'
]                          
                          
X=df[feats].values  
#pojedyncze [] to vector a [[]] to macierz, przekazanie wartości do macierzy
#po przecinku moze byc wiecej kolumn

y=df['delay_secs'].values #opznienie jako vector
model = DecisionTreeRegressor(max_depth=10,random_state=0)
#skąd wiadomo jaki dać max deph ?, zwykle sie zostaje na 10
#im wieksza wartosc tym mocniej sie nauczy, ale trzeba uwazac na przeuczenie, 
#przestanie być opisowy a zacznie byc szczegółowy i przestanie rozpoznawać, 
#bedzie musiał dostać bardziej szczegółowe dane
####model = RandomForestRegressor(max_depth=10,n_estimators=50,n_jobs=4,random_state=0)
#n_jobs=4 ilosc rdzenie w procesorze 

scores = cross_val_score(model,X,y,cv=3, scoring='neg_mean_absolute_error') 
#cv to fault, ta liczba dzieli zbiór uczący na 
#tyle np 100/5 to po 20 dla każdej iteracji tworzenia modeli
#te 20 jest przesuwanych w trakcie każdej iteracji o +20, 
#ta część jest w każdym zbiorze testowym..?

#neg_mean_absolute_error -absolute - wartość absolutna(konwertuje na dodatnia)
#od wartości pred odejmuje sie test i tak wychodzi error (różnica)
#mean - wartosc srednia (z całego wiersza)
#neg - zeby zamienić wynik na ujemny żeby był poniżej zera, 
#równe zero to najlepszy wynik

np.mean(scores), np.std(scores)
# srednia i różnica w wynikach w sekundach


In [ ]:
X

In [ ]:
df['number']